In [ ]:
import torch
import utils
import numpy as np
from Vocoder import Vocoder
from Model import Model
import torchaudio as ta
from resemblyzer import preprocess_wav, VoiceEncoder


#load Model
myModel = Model()

#load desired model with trained weights
myModel.load_state_dict(torch.load('model_final_e2_w_style.pt'))

# Create this folder manually if you don't have it
temp_dir = "./temp"



In [ ]:
#Split Song into vocals and instrumentals

from audio_separator.separator import Separator



# Initialize the Separator class (with optional configuration properties, below)
separator = Separator(output_format="mp3", output_dir=temp_dir)

# Load a machine learning model (if unspecified, defaults to 'model_mel_band_roformer_ep_3005_sdr_11.4360.ckpt')
separator.load_model(model_filename='UVR-MDX-NET-Inst_HQ_3.onnx')

song_path = r"C:\Users\baseb\Downloads\Not Like Us.mp3" #put the path to your song here. r here for raw string to prevent escape characters when copying fpath


output_files = separator.separate(song_path,)
print(f"Separation complete! Output file(s): {' '.join(output_files)}")


print('Successfully Separated Vocals and Instrumentals!')




In [15]:
vocal_file = os.path.join(temp_dir,output_files[0])
instrumental_file = os.path.join(temp_dir,output_files[1])

In [16]:
#make the spec of song 
#optionally start here if you already have the song separated


path_to_song = vocal_file
transform = ta.transforms.Spectrogram(510)

vocals = ta.load(path_to_song)
mono = torch.mean(vocals[0],0,False)

spec = transform(mono)
spec = torch.unsqueeze(spec,0)
print(spec.shape)


torch.Size([1, 256, 47289])


In [17]:
#Make style of song
wav = preprocess_wav(path_to_song)
encoder = VoiceEncoder()
style = encoder.embed_utterance(wav)

style = torch.tensor(style)
style = torch.unsqueeze(style,0)
print(style.shape)


Loaded the voice encoder model on cuda in 0.02 seconds.
torch.Size([1, 256])


In [18]:
#Get the input voice's style embedding

#path_to_targets = [r"C:\Users\baseb\OneDrive\Desktop\Spring 2024 Classes\DL\Project\Voice-to-Song-DL-Final\Tracks_2\vocals_chunks\01 Cherry_(Vocals)_UVR-MDX-NET-Inst_HQ_3-chunk-42.mp3",
# r"C:\Users\baseb\OneDrive\Desktop\Spring 2024 Classes\DL\Project\Voice-to-Song-DL-Final\Tracks_2\vocals_chunks\01 Cherry_(Vocals)_UVR-MDX-NET-Inst_HQ_3-chunk-39.mp3",
# r"C:\Users\baseb\OneDrive\Desktop\Spring 2024 Classes\DL\Project\Voice-to-Song-DL-Final\Tracks_2\vocals_chunks\01 Cherry_(Vocals)_UVR-MDX-NET-Inst_HQ_3-chunk-40.mp3",
# r"C:\Users\baseb\OneDrive\Desktop\Spring 2024 Classes\DL\Project\Voice-to-Song-DL-Final\Tracks_2\vocals_chunks\01 Cherry_(Vocals)_UVR-MDX-NET-Inst_HQ_3-chunk-41.mp3"]
path_to_targets = [r"C:\Users\baseb\OneDrive\Desktop\Spring 2024 Classes\DL\Project\Voice-to-Song-DL-Final\Tracks_2\Speech_examples\Jerome_style.m4a"
,r"C:\Users\baseb\OneDrive\Desktop\Spring 2024 Classes\DL\Project\Voice-to-Song-DL-Final\Tracks_2\Speech_examples\style_jerome2.m4a"]
embeddings = []

style_name = 'jerome' #put a name here to denote the style of the singer

for path in path_to_targets:
    _wav = preprocess_wav(path)
    t = encoder.embed_utterance(_wav)
    embeddings.append(t)


embeddings = np.array(embeddings)
print(embeddings.shape)
embeddings = np.mean(embeddings,0)
print(embeddings.shape)

target = torch.tensor(embeddings)
target = torch.unsqueeze(target,0)

print(target.shape)


(2, 256)
(256,)
torch.Size([1, 256])


In [20]:
myModel.eval()
outputs = myModel.forward((spec,style,target))

waveforms = outputs[3] #get the postnet output




In [21]:
import torch
import numpy as np
import os
import soundfile as sf
from Vocoder import Vocoder

if __name__ == "__main__":
    # Example batch of magnitude spectrograms of shape (N, 80, T)
    
    
    magnitude_spectrogram_batch = torch.detach(waveforms)
    magnitude_spectrogram_batch= magnitude_spectrogram_batch.to('cuda')
    print(magnitude_spectrogram_batch.shape)
    # Instantiate Vocoder
    vocoder = Vocoder(n_fft=510, hop_length=255, win_length=510, num_iters=50)

    # Output directory to save reconstructed waveforms
    output_dir = temp_dir

    # Reconstruct waveforms from the batch of magnitude spectrograms
    reconstructed_waveforms = vocoder.reconstruct(magnitude_spectrogram_batch, output_dir, sample_rate=44100,save = True)

    # Print the shape of the reconstructed waveforms
    print(reconstructed_waveforms.shape)

    

torch.Size([1, 256, 47289])
torch.Size([1, 12058440])


In [22]:
from utils import overlay_audio

path_to_new_voice = os.path.join(output_dir,'reconstructed_0.wav')



out_path_and_file_name = './jerome_drake_style/Song Outputs/notLikeUs_Jerome.mp3' #put your output path and file name here example: ./jerome_drake_style/Song Outputs/rap_god_jerome.mp3

overlay_audio(path_to_new_voice,instrumental_file,out_path_and_file_name)

